In [ ]:
import pandas as pd
import numpy as np 
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn import cross_validation
from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn.externals import joblib
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import cross_validation

In [ ]:
mapping=pd.read_csv("train/cat_amazon_map.tsv",sep='\t',names=["unmapped","mapped"])
pd.options.display.max_colwidth=50

In [ ]:
brand_dfs=[]
for chunk in pd.read_csv("train/amazon_train.csv",chunksize=10000):
    chunk["top"]=chunk.breadCrumbs.str.lower().apply(lambda l: l.split(">")[0].replace("-"," ").rstrip())
    df=chunk.merge(mapping,how="inner",left_on="top",right_on="unmapped").ix[:,["brandText","mapped"]]
    brand_dfs.append(df)

brand_df=pd.concat(brand_dfs)
del brand_dfs
brand_df['brandText']=brand_df.brandText.str.lower()

#brand_space=brand_df.groupby('brandText')['mapped'].apply(lambda l:list(set(list(l))))
vect=CountVectorizer(stop_words='english',lowercase=True)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(brand_df.brandText, brand_df.mapped, test_size=0.1, random_state=0)

In [ ]:
X=vect.fit_transform(X_train.replace(np.nan,""))
mlb=MultiLabelBinarizer()
y=mlb.fit_transform(y_train)
del brand_space
del brand_df

classif = OneVsRestClassifier(linear_model.LogisticRegression(fit_intercept=False))
ovr=classif.fit(X, y)
classif.classes_=mlb.classes_

brand_classifier = Pipeline([('output_transform', mlb),('input_transform',vect),('classifier', classif)])
#joblib.dump(brand_classifier,'brand_toplevel_classifier.pkl')

In [ ]:
y_hat=X_test.to_series().apply(lambda x:[i for i,j in zip(mlb.classes_, classif.predict_proba(vect.transform([x.lower()]))[0]) if j>0.4][:8])

In [ ]:
joblib.dump(brand_classifier,'brand_toplevel_classifier_row.pkl')